## Foundations of Deep Learning 24/25 // Data Science MSc - unimib

### Temporal Convolutional Networks and the Tennessee Eastman Process dataset 

### TCN Autoencoders for time-series anomaly detection 

 E. Mosca - 925279

#### *Introduction*

In this work, I have implemented TCNAE-based models in Tf/Keras, the main inspiration for this was the following paper from Thill et al. (https://www.sciencedirect.com/science/article/abs/pii/S1568494621006724?fr=RR-2&ref=pdf_download&rr=95139c04af9dee8c)

TCNAEs are again similar in concept to Autoencoder architectures using usual 2D convolutions, but characterised by 1D dilated causal convolutions just like TCNs, and have the latter as their building blocks. From our classification part, we already have an idea of which hyperparameters have worked well with this type of data and TCNs, and we already know the receptive field we need(based on the training set). With that said a lot of hyperparameters will be the same as the TCN model in the project.

In this implementation:
- The autoencoder is composed of an encoder(a single tcn block) and a decoder(a single tcn block, characterised by mirrored dilations)
- The encoder is a series of dilated causal convolutions with dilation factors being powers of 2, after each convolution we use a 1x1 convolution to downsample the channels dimension, and save every one of these "intermediate" representations along the way.
- A characteristic of the architecture is that we (batch) norm, (relu) activate and (spatial) dropout after each of the convolutions in the encoder(and decoder also) except for the 1x1 convolutions, where we are trying to learn an identity mapping(linear activation is used)
- before the bottleneck, all intermediate convolutions are concatenated along the channel axis(inception style) this gives the model a different way of looking at the data at different time resolutions. Then, we downsample said concatenation via another 1x1 convolution, and we spatially downsample using 1D average pooling(using a window called sample_rate), this brings us to our shrunk bottleneck
- The decoder takes as input said bottleneck, upsamples it spatially by repeating each time point 'sample_rate-times', then it restores the pre-concatenation channel dimension via 1x1 convolution.
- From here, the decoder simply repeats whats done by the encoder in reverse. Note: we still save intermediate features after each dilated convolution in the decoder, yet we concatenate them right before the final layer, so the decoder is not exactly a mirrored version of the decoder.
- The decoder's intermediate feature concatenation is fed to a 1x1 again, restoring the original input channel dimension

We still use L2 regularizers, glorot_uniform initializers, kernel size of 3, dropout rate of 0.1, same dilations for a good receptive field.

Throughout the encoder and decoder, each dilated convolution is done with the same number of filters, and each 1x1 convolution always shrinks the dimensions down to the same dimension. Ither parameters are the sample rate and the dimension of the bottleneck.

In [2]:
from tensorflow import keras 
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, SpatialDropout1D, Add, Dense, AveragePooling1D, Concatenate, UpSampling1D
from tensorflow.keras.models import Model
from keras.regularizers import L2

In [ ]:
# Dilated conv params
kernel_size = 3
num_filters = 64
dilations = [1, 2, 4, 8, 16, 32, 64,128]
dropout_rate = 0.1
# 1x1 conv params
shrunk_dim = 32
# Pooling params
bottleneck_dim = 16
sample_rate = 10

num_features = 52

In [ ]:
def dilated_conv(x, dilation_rate, filters=num_filters, kernel_size=kernel_size, dropout_rate=dropout_rate):
    x = Conv1D(filters, kernel_size, dilation_rate=dilation_rate,
               padding='causal', kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = SpatialDropout1D(dropout_rate)(x)
    return x

def onexone_conv(x, filters, kernel_size=1):
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation='linear')(x)
    return x

def encoder_layers(x, sample_rate=sample_rate, bottleneck_dim=bottleneck_dim):
    shrunk_convs = []
    for dilation in dilations:
        x = dilated_conv(x, dilation_rate=dilation)
        shrunk_convs.append(onexone_conv(x,filters=shrunk_dim))
    x = Concatenate(axis=-1)(shrunk_convs)
    x = onexone_conv(x, filters=bottleneck_dim)
    x = AveragePooling1D(pool_size=sample_rate)(x)
    return x

def decoder_layers(x, input_shape, sample_rate=sample_rate):
    x = UpSampling1D(size=sample_rate)(x)
    shrunk_convs = []
    for dilation in reversed(dilations):
        x = dilated_conv(x, dilation_rate=dilation)
        shrunk_convs.append(onexone_conv(x,filters=shrunk_dim))
    x = Concatenate(axis=-1)(shrunk_convs)
    x = onexone_conv(x, filters=input_shape[-1])
    return x

def build_tcn_ae_v0(input_shape=(None,num_features), sample_rate=sample_rate, bottleneck_dim=bottleneck_dim):
    inputs = Input(batch_shape=(None, input_shape[0], input_shape[1]))
    x = encoder_layers(inputs, sample_rate, bottleneck_dim)
    outputs = decoder_layers(x, input_shape, sample_rate)
    model = Model(inputs, outputs)
    return model


In [3]:
model00 = tf.keras.models.load_model("TCNAE00.keras")

In [4]:
model00.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, None, 52)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_102 (Conv1D) │ (None, None, 64)  │     10,048 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None, 64)  │        256 │ conv1d_102[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_48 (ReLU)     │ (None, None, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_… │ (None, None, 64)  │          0 │ re_lu_48[0][0]    │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_104 (Conv1D) │ (None, None, 64)  │     12,352 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None, 64)  │        256 │ conv1d_104[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_49 (ReLU)     │ (None, None, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_… │ (None, None, 64)  │          0 │ re_lu_49[0][0]    │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_106 (Conv1D) │ (None, None, 64)  │     12,352 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None, 64)  │        256 │ conv1d_106[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_50 (ReLU)     │ (None, None, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_… │ (None, None, 64)  │          0 │ re_lu_50[0][0]    │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_108 (Conv1D) │ (None, None, 64)  │     12,352 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None, 64)  │        256 │ conv1d_108[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_51 (ReLU)     │ (None, None, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_… │ (None, None, 64)  │          0 │ re_lu_51[0][0]    │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_110 (Conv1D) │ (None, None, 64)  │     12,352 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None, 64)  │        256 │ conv1d_110[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_52 (ReLU)     │ (None, None, 64)  │          0 │ batch_normalizat

 Total params: 718,798 (2.74 MB)

 Trainable params: 238,916 (933.27 KB)

 Non-trainable params: 2,048 (8.00 KB)

 Optimizer params: 477,834 (1.82 MB)